In [1]:
!pip install --user scikit-learn
!pip install --user tensorflow


In [2]:
import sqlite3
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
import os
import numpy as np

In [3]:
# loading data

# Create connection
con = sqlite3.connect('uah-usd-data.db')

df = pd.read_sql_query("SELECT * FROM data_for_model", con)

In [4]:
df.head()

,quote_date,uah_usd,eur_usd,rub_usd,pln_usd,brent,steal,wheat,mvemsd
0,2016-01-04,24.000667,0.9152,72.8817,3.8807,37.22,327.0,4.5825,1381.20
1,2016-01-05,24.000667,0.9152,72.8817,3.8807,36.42,327.0,4.6125,1381.95
2,2016-01-06,23.783691,0.9306,72.9293,4.0093,34.23,327.0,4.6275,1382.05
3,2016-01-07,23.783691,0.9306,72.9293,4.0093,33.75,327.0,4.6850,1379.97
4,2016-01-08,23.783691,0.9306,72.9293,4.0093,33.55,327.0,4.7850,1378.09


In [5]:
df.shape

(1342, 9)

In [6]:
df.describe()

,uah_usd,eur_usd,rub_usd,pln_usd,brent,steal,wheat,mvemsd
count,1342.000000,1342.000000,1342.000000,1342.000000,1342.000000,1342.000000,1342.000000,1342.000000
mean,26.537279,0.877764,65.771708,3.810266,56.038361,475.310171,4.982199,1677.446975
std,1.214696,0.035860,6.280954,0.181634,13.185902,81.362388,0.703057,150.547926
min,23.266758,0.800400,55.671200,3.324400,9.120000,318.810000,3.806500,1357.960000
25%,25.784091,0.848700,61.584050,3.712325,47.320000,421.000000,4.430625,1577.805000
50%,26.604243,0.882800,64.746650,3.807250,56.290000,457.000000,4.906250,1638.440000
75%,27.629569,0.902150,70.276800,3.923650,65.657500,521.750000,5.272500,1815.187500
max,28.875837,0.964900,81.839600,4.291900,86.070000,794.000000,7.617500,1983.100000


In [7]:
df.iloc[:, 1]

0       24.000667
1       24.000667
2       23.783691
3       23.783691
4       23.783691
          ...    
1337    27.526000
1338    27.467400
1339    27.438100
1340    27.344900
1341    27.340000
Name: uah_usd, Length: 1342, dtype: float64

In [8]:
col = df.pop("uah_usd")
df.insert(8, "uah_usd", col)
df.head()

,quote_date,eur_usd,rub_usd,pln_usd,brent,steal,wheat,mvemsd,uah_usd
0,2016-01-04,0.9152,72.8817,3.8807,37.22,327.0,4.5825,1381.20,24.000667
1,2016-01-05,0.9152,72.8817,3.8807,36.42,327.0,4.6125,1381.95,24.000667
2,2016-01-06,0.9306,72.9293,4.0093,34.23,327.0,4.6275,1382.05,23.783691
3,2016-01-07,0.9306,72.9293,4.0093,33.75,327.0,4.6850,1379.97,23.783691
4,2016-01-08,0.9306,72.9293,4.0093,33.55,327.0,4.7850,1378.09,23.783691


In [9]:
df.iloc[:,0:9].corr()

,eur_usd,rub_usd,pln_usd,brent,steal,wheat,mvemsd,uah_usd
eur_usd,1.000000,-0.108650,0.852911,-0.362860,-0.770225,-0.435740,-0.411130,-0.441139
rub_usd,-0.108650,1.000000,0.337671,-0.428241,-0.001250,0.647963,0.382909,0.326369
pln_usd,0.852911,0.337671,1.000000,-0.535269,-0.607274,-0.073607,-0.156610,-0.216208
brent,-0.362860,-0.428241,-0.535269,1.000000,0.575473,0.086570,0.122574,0.075646
steal,-0.770225,-0.001250,-0.607274,0.575473,1.000000,0.527761,0.435221,0.458621
wheat,-0.435740,0.647963,-0.073607,0.086570,0.527761,1.000000,0.711088,0.353578
mvemsd,-0.411130,0.382909,-0.156610,0.122574,0.435221,0.711088,1.000000,0.216772
uah_usd,-0.441139,0.326369,-0.216208,0.075646,0.458621,0.353578,0.216772,1.000000


In [10]:
X = df.iloc[:, 1:-1].values
Y = df.iloc[:, -1].values

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

In [12]:
#data preprocessing
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [13]:
X_train.shape

(1073, 7)

In [14]:
model = Sequential()

#first hidden layer
model.add(Dense(64, input_dim=7, activation='relu'))
model.add(BatchNormalization())

#second hidden layer
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())

#third hidden layer
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
#model.add(Dropout(0.5))

#output layer
model.add(Dense(1, activation='linear'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                512       
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0

In [16]:
#learning rate 0.001
model.compile(loss='mean_squared_error', optimizer='nadam',)

In [17]:
output_dir = 'model_output/'
run_name = 'my_model'
#run_name = 'my_model_dropuot_0.5'
output_pass = output_dir + run_name

In [ ]:
if not os.path.exists(output_pass):
    os.makedirs(output_pass)

In [ ]:
modelcheckpoint = ModelCheckpoint(output_pass + '/weights.{epoch:02d}.hdf5', save_weights_only = True)

In [ ]:
es_callback = EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

In [ ]:
tensorboard = TensorBoard(log_dir='logs/' + run_name)

In [ ]:
 model.fit(X_train, y_train, batch_size = 24, epochs=50, verbose=1, validation_data=(X_test, y_test),
          callbacks=[modelcheckpoint, es_callback, tensorboard])


In [ ]:
#specify number after training
model.load_weights(output_pass + '/weights.29.hdf5')

In [ ]:
y_test[268]

In [ ]:
#prediction
model.predict(np.reshape(X_test[268], [1, 7]))